In [1]:
import sympy as sym
import helpers
from IPython.display import display
from IPython.display import Code

## ST-VK Hyperelastic Model

In [2]:
# Define input variables and constants.
Fv = sym.MatrixSymbol('Fv', 9, 1)
lame1, lame2 = sym.symbols('lame1 lame2', constants = True)

# Define expression to evaluate.
F = sym.Matrix([[ Fv[0], Fv[1], Fv[2] ], [ Fv[3], Fv[4], Fv[5] ], [ Fv[6], Fv[7], Fv[8] ]])
I = sym.eye(3)
E1 = (F.T * F - I) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit() # Explicitly evaluate trace. Sympy has problems differentiating
trE2 = sym.Trace(E2).doit() # it otherwise.

U = lame1 * (trE1 ** 2) / 2 + lame2 * trE2
dUdF = helpers.gradient(U, [ *Fv ])
d2UdF2 = helpers.hessian(U, [ *Fv ])

# Define specific routines to export to code.
export = [("U", U), ("dUdF", dUdF), ("d2UdF2", d2UdF2)]

# Generate code and save to disk.
code = helpers.generate(export)
helpers.save('generated/stvk-U.inc', code[0])
helpers.save('generated/stvk-dUdF.inc', code[1])
helpers.save('generated/stvk-d2UdF2.inc', code[2])

# Display.
#print('Energy')
#display(U)
#print('Gradient')
#display(dUdF)
#print('Hessian')
#display(d2UdF2)


## ST-VK Dissipation Potential

In [3]:
# Define input variables and constants.
Fv = sym.MatrixSymbol('Fv', 9, 1)
Gv = sym.MatrixSymbol('Gv', 9, 1)
lame1, lame2, beta = sym.symbols('lame1 lame2 beta', constants = True)

# Define expression to evaluate.
F = sym.Matrix([[ Fv[0], Fv[1], Fv[2] ], [ Fv[3], Fv[4], Fv[5] ], [ Fv[6], Fv[7], Fv[8] ]])
G = sym.Matrix([[ Gv[0], Gv[1], Gv[2] ], [ Gv[3], Gv[4], Gv[5] ], [ Gv[6], Gv[7], Gv[8] ]])
I = sym.eye(3)
E1 = (G.T * F + F.T * G) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit()
trE2 = sym.Trace(E2).doit()

R = beta * lame1 * (trE1 ** 2) / 2 + beta * lame2 * trE2
dRdFG = helpers.gradient(R, [ *Fv, *Gv ])
d2RdFG2 = helpers.hessian(R, [ *Fv, *Gv ])

# Define specific routines to export to code.
export = [("R", R), ("dRdFG", dRdFG), ("d2RdF2", d2RdFG2)]

# Generate code and save to disk.
code = helpers.generate(export)
helpers.save('generated/stvk-R.inc', code[0])
helpers.save('generated/stvk-dRdFG.inc', code[1])
helpers.save('generated/stvk-d2RdFG2.inc', code[2])

# Display.
#print('Dissipation potential')
#display(R)
#print('Gradient')
#display(dRdFG)
#print('Hessian')
#display(d2RdFG2)